In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# Import necessary libraries.
import os
import sys
import numpy as np
import pandas as pd
import json
from pprint import pprint
import requests
import time
import random
!pip install langdetect
from langdetect import detect

from datetime import datetime, date, timedelta
import string
from scipy import stats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
def date2timestamp(date):
    '''
    "yyyy/mm/dd"
    '''
    return int(time.mktime(datetime.strptime(date, "%Y/%m/%d").timetuple()))

def next_day(date):
  tmrw = datetime.strptime(date, "%Y/%m/%d") + timedelta(days=1)
  return tmrw.strftime('%Y/%m/%d')

def list_of_days(start_date, end_date):
    '''
    start_date = date(2019, 1, 19)   # start date
    end_date = date(2019, 3, 22)   # end date
    '''
    delta = end_date - start_date       # as timedelta
    days = []
    for i in range(delta.days + 1):
        day = start_date + timedelta(days=i)
        days.append(str(day).replace('-', '/'))
    return days

#print(list_of_days(date(2019, 1, 19), date(2019, 3, 22) ))

In [19]:
# output dir for save data
output_dir = '/content/drive/My Drive/Colab Notebooks/Suicide and Depression/data/input/'

In [28]:
# Method used to get posts
def scrape_reddit(output_dir, subreddit, timeframe, date_start, date_end, size = 1000):
    '''
    size = {1,1000} #amount of posts
    '''
    start = date2timestamp(date_start)
    end = date2timestamp(date_end)
    # use the pushshift api to extract out data
    url = 'https://api.pushshift.io/reddit/search/submission/?subreddit={}&sort=created_utc&sort_type=created_utc&after={}&before={}&size={}'.format(subreddit,start, end, size)
    print(url)
    try:
        posts = requests.get(url)
        posts = posts.json()
        print(posts)
        posts = posts['data']
    #except (ConnectionError, JSONDecodeError):    
    except:
        time.sleep(30)
        posts = requests.get(url)
        posts = posts.json()
        posts = posts['data']
    df = pd.DataFrame(columns=['date', 'author', 'title', 'selftext', 'subreddit'])
    
    for post in posts:
      if 'selftext' in post: 
        text = post['selftext']
        try: 
          if detect(text) == 'en': # check if text is in english - if the language detected is not in the langdetect library, then continue to the next post
            df = df.append({'date': date_start, 'author': post['author'], 'title': post['title'], 'selftext': post['selftext'], 'subreddit': subreddit}, ignore_index=True)
        except:
          continue
    return df

In [21]:

# Subreddits to scrape:

subreddits = ['suicidewatch','depression']

In [22]:
days = list_of_days(date(2022,12,1), date(2022,12,5))
print(days)

['2022/12/01', '2022/12/02', '2022/12/03', '2022/12/04', '2022/12/05']


In [29]:
size = 1000

timeframe = 'pre'

for subreddit in subreddits:
  subreddit_df = pd.DataFrame(columns=['date', 'author', 'title', 'selftext', 'subreddit'])
  days_local = list(days)
  while subreddit_df.shape[0] < 50000 and days_local:
    idx = random.randint(0, len(days_local)-1)
    date_start = days_local.pop(idx)
    date_end = next_day(date_start)
    df = scrape_reddit(output_dir, subreddit, timeframe, date_start, date_end, size = size)
    subreddit_df = pd.concat([subreddit_df, df])
    time.sleep(0.5)
    print(subreddit)
    print(subreddit_df.shape)
  subreddit_df.to_csv(os.path.join(output_dir, '{}_{}_small_new.csv'.format(subreddit, timeframe)), index=False)